In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
from keras.datasets import mnist

In [5]:
(X_train, y_train), (X_test, y_test) = mnist.load_data() 

In [6]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


# Split the MNIST data set 

In [8]:
pt_x_train = []
pt_y_train = []
pt_x_test = []
pt_y_test = []

tl_x_train = []
tl_y_train = []
tl_x_test = []
tl_y_test = []


we r only working with 20k dataste 

In [9]:
m = 20000
for i in range(m):
    if y_train[i]<5:
        pt_x_train.append(X_train[i]/255)
        pt_y_train.append(y_train[i])
    else:
        tl_x_train.append(X_train[i]/255)
        tl_y_train.append(y_train[i])
        
mm = 10000
for i in range(mm):
    if y_test[i]<5:
        pt_x_test.append(X_test[i]/255)
        pt_y_test.append(y_test[i])
    else:
        tl_x_test.append(X_test[i]/255)
        tl_y_test.append(y_test[i])
    

In [10]:
pt_x_train = np.asarray(pt_x_train).reshape(-1,28,28,1)
pt_x_test = np.asarray(pt_x_test).reshape(-1,28,28,1)

tl_x_train = np.asarray(tl_x_train).reshape(-1,28,28,1)
tl_x_test = np.asarray(tl_x_test).reshape(-1,28,28,1)

In [11]:
print(pt_x_train.shape, tl_x_train.shape)
print(pt_x_test.shape, tl_x_test.shape)

(10225, 28, 28, 1) (9775, 28, 28, 1)
(5139, 28, 28, 1) (4861, 28, 28, 1)


In [16]:
from keras.utils import np_utils


In [17]:
pt_y_train = np_utils.to_categorical(np.asarray(pt_y_train))
pt_y_test = np_utils.to_categorical(np.asarray(pt_y_test))

tl_y_train = np_utils.to_categorical(np.asarray(tl_y_train))
tl_y_test = np_utils.to_categorical(np.asarray(tl_y_test))



In [18]:
pt_y_train[:5]

array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.]], dtype=float32)

# Pretrend Model

In [19]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, Flatten,MaxPool2D,Dropout

In [23]:
model = Sequential()

model.add(Conv2D(32,(5,5),
                input_shape = (28,28,1),
                activation = 'relu'))

model.add(Conv2D(16,(5,5),
                activation='relu'))

model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(8,(3,3),
                activation='relu'))

model.add(Flatten())

model.add(Dropout(0.4))

model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 20, 16)        12816     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 8)           1160      
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)              

#TRaining

In [24]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [25]:
model.fit(pt_x_train, pt_y_train,
         validation_data=(pt_x_test, pt_y_test),
         batch_size=16,
         epochs=2,
         shuffle=True,
         verbose=2)

Train on 10225 samples, validate on 5139 samples
Epoch 1/2
 - 23s - loss: 0.1601 - accuracy: 0.9476 - val_loss: 0.0259 - val_accuracy: 0.9907
Epoch 2/2
 - 23s - loss: 0.0552 - accuracy: 0.9826 - val_loss: 0.0146 - val_accuracy: 0.9955


In [26]:
model.layers

In [27]:
for layer in model.layers[:6]:
    layer.trainable=False
for layer in model.layers:
    print(layer.trainable)

False
False
False
False
False
False
True
True
True
True


In [29]:
tl_model = Sequential(model.layers[:6])

tl_model.add(Dense(128))
tl_model.add(Activation('relu'))

tl_model.add(Dense(10))
tl_model.add(Activation('softmax'))

tl_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 20, 16)        12816     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 8)           1160      
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)              

# Result

In [30]:
tl_model.compile(loss = 'categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

In [34]:
import datetime
start = datetime.datetime.now()
tl_model.fit(tl_x_train, tl_y_train,
            validation_data=(tl_x_test, tl_y_test),
            epochs = 2,
            verbose=2,
            batch_size=16)

end = datetime.datetime.now()
print(end - start)

Train on 9775 samples, validate on 4861 samples
Epoch 1/2
 - 8s - loss: 0.0761 - accuracy: 0.9750 - val_loss: 0.0368 - val_accuracy: 0.9870
Epoch 2/2
 - 8s - loss: 0.0651 - accuracy: 0.9797 - val_loss: 0.0401 - val_accuracy: 0.9860
0:00:16.203961
